In [2]:
pip install tqdm


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications.resnet import preprocess_input
from tqdm import tqdm  # Add progress bar

# Load grayscale fingerprint data
X = np.load("X_fingerprint_segmented.npy")
y = np.load("y_fingerprint_segmented.npy")

# Load ResNet101 model
resnet_model = ResNet101(weights='imagenet', include_top=False, pooling='avg', input_shape=(224, 224, 3))

# Function to process in batches with progress
def extract_features_in_batches(X, batch_size=128):
    features = []
    num_batches = len(X) // batch_size + int(len(X) % batch_size != 0)

    for i in tqdm(range(0, len(X), batch_size), desc="🔄 Extracting Features"):
        batch = X[i:i+batch_size]
        batch = np.repeat(batch[..., np.newaxis], 3, axis=-1)
        batch_resized = tf.image.resize(batch, (224, 224)).numpy()
        batch_preprocessed = preprocess_input(batch_resized)
        batch_features = resnet_model.predict(batch_preprocessed, verbose=0)
        features.append(batch_features)

    return np.concatenate(features, axis=0)

# Extract with visual feedback
features = extract_features_in_batches(X, batch_size=128)
print(f"✅ Extracted features shape: {features.shape}")


🔄 Extracting Features: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 188/188 [1:52:46<00:00, 35.99s/it]


✅ Extracted features shape: (24000, 2048)


In [6]:
np.save("features.npy", features)
np.save("labels.npy", y)
